In [4]:
import warnings
warnings.filterwarnings('ignore')
from astropy.io import fits
import os
from astropy.table import join
from glob import glob 
import sys
sys.path.append('./')
from modules import dendro_dendro, dendro_misc, dendro_props, dendro_mask

In [5]:
# Define names and filenames...

galaxy = 'ngc0628'
galaxy_hst = 'ngc628e'
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies'

hstha_file = '%s/data_hstha/%s/hst_contsub/%s_hst_ha_sic.fits' %(root_dir, galaxy_hst, galaxy_hst)
muscat_file = '%s/data_hstha/%s/muse/%s_nebmask.fits' %(root_dir, galaxy_hst, galaxy.upper())
musha_file = '%s/data_hstha/%s/muse/%s-*_MAPS.fits' %(root_dir, galaxy_hst, galaxy.upper())
musha_file = glob(musha_file)[0] #because of resolution in name

cutout_dir = '%s/data_hstha_nebulae_catalogue/%s/cutouts' %(root_dir, galaxy_hst)
dendro_dir = '%s/data_hstha_nebulae_catalogue/%s/dendro' %(root_dir, galaxy_hst)
cutouts_hdus_dir = '%s/data_hstha_nebulae_catalogue/%s/cutouts_hdus' %(root_dir, galaxy_hst)

rerun_all = True
rerun_masking = True

regions_file = '%s/sample.reg' %cutout_dir
regions_pickel_file = '%s/sample.pickel' %cutout_dir
sample_table_file = '%s/data_misc/sample_table/phangs_sample_table_v1p6.fits' %root_dir
muscat_table_file = '%s/data_misc/nebulae_catalogue/Nebulae_catalogue_v3.fits' %root_dir

print(hstha_file)
print(muscat_file)
print(musha_file)
print(cutout_dir)
print(dendro_dir)
print(cutouts_hdus_dir)
print(regions_file)
print(regions_pickel_file)
print(sample_table_file)
print(muscat_table_file)

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc628c/hst_contsub/ngc628c_hst_ha_sic.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc628c/muse/NGC0628_nebmask.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc628c/muse/NGC0628-0.92asec_MAPS.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc628c/cutouts
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc628c/dendro
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc628c/cutouts_hdus
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc628c/cutouts/sample.reg
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha_nebulae_catalogue/ngc628c/cutouts/sample.pickel
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/sample_table/phangs_sample_table_v1p6.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/nebulae_catalogue/Nebu

In [ ]:
# Load regions, sample table and HDUs... 

hdus_cutouts = dendro_misc.load_pickle('%s/hdus_all.pickel' %cutout_dir)
regions = dendro_misc.load_pickle(regions_pickel_file)

sample_table = dendro_misc.get_galaxyprops(galaxy, sample_table_file)
muscat_table = dendro_misc.get_museprops(galaxy, muscat_table_file)

In [ ]:
# Load cutout hdus with smoothed, masked, and non-masked data...

hdus_file = '%s/hdus_all_withmasked.pickel' %cutout_dir
muscat_regionIDs_file =  '%s/muscat_regionIDs.pickel' %cutout_dir

if os.path.exists(hdus_file) & ~rerun_masking:
    muscat_regionIDs = muscat_table['region_ID']
    hdus = dendro_misc.load_pickle(hdus_file)
else: 
    muscat_regionIDs = muscat_table['region_ID']
    hdus = dendro_dendro.get_maskedhdus(hdus_cutouts, regions, muscat_regionIDs)
    dendro_misc.save_pickle(hdus, hdus_file)

In [ ]:
# Get dendrogram for all... 

props_all, hdus = dendro_dendro.get_dedro_all(hdus, sample_table, muscat_regionIDs)

In [ ]:
# Add additional infomation from MUSE and save... 

props_all_muscat_table = join(props_all, muscat_table, keys='region_ID')

props_all_muscat_table['flux_corr'] = dendro_props.correct_ha_flux(props_all_muscat_table)
props_all_muscat_table['ha_lum_hst'] = dendro_props.calculate_luminosity(props_all_muscat_table['flux_corr']*1e-20, sample_table['dist'].quantity[0])
    
props_all_muscat_table.rename_column('flux', 'HA6562_FLUX_HST')
props_all_muscat_table.rename_column('flux_corr', 'HA6562_FLUX_HST_CORR')
props_all_muscat_table.rename_column('ha_lum_hst', 'HA6562_LUMINOSITY_HST')

dendro_misc.save_pickle(hdus, '%s/hdus_all_withmasked_withdendro.pickel' %dendro_dir)
dendro_misc.save_pickle(props_all_muscat_table, '%s/props_all.pickel' %dendro_dir)        
props_all_muscat_table.write('%s/props_all.fits' %dendro_dir, overwrite=True)

In [ ]:
# Creat masks and region files... 

# Save HDUs as HDU lists for each region with RegionID
dendro_dendro.get_hdulists(hdus, muscat_regionIDs, outputdir=cutouts_hdus_dir)

# Save region file of masks 
dendro_mask.get_ds9regions_circ_decor(props_all, '%s/%s_trunk_regions.reg' %(dendro_dir, galaxy))
dendro_mask.get_ds9regions_ellip_decor(props_all, '%s/%s_sigma_regions.reg' %(dendro_dir, galaxy))

# Save the mask for whole mosaic
hstha_hdu = fits.open(hstha_file)[0]
dendro_mask.get_hdumask(hstha_hdu, hdus['indexmap_trunk_hdu'], '%s/%s_trunk_mask.fits'  %(dendro_dir, galaxy))
# dendro_mask.get_hdumask(hstha_hdu, hdus['indexmap_trunk_close_hdu'], '%s/%s_trunk_close_mask.fits'  %(dendro_dir, galaxy))

In [ ]:
# Save flags for when regions are touching
hdu_mask = fits.open('%s/%s_trunk_mask.fits'  %(dendro_dir, galaxy))[0]
props_all_muscat_table = dendro_dendro.get_flag_touch(props_all_muscat_table, hdu_mask)
props_all_muscat_table.write('%s/props_all.fits' %dendro_dir, overwrite=True)